# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [ ]:
from pyspark.sql import SparkSession
import os

data_dir = '../../data/Spark_ML/Clustering/'

In [ ]:
spark = SparkSession.builder.appName('cluster_consulting_projecg').getOrCreate()

data = spark.read.csv(os.path.join(data_dir, 'hack_data.csv'), inferSchema=True, header=True)

data.printSchema()

## Format data for MLlib

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

col_names = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 
             'Pages_Corrupted', 'WPM_Typing_Speed']

assembler = VectorAssembler(inputCols=col_names, outputCol='features')
final_data = assembler.transform(data)

scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(final_data)
final_data = scaler_model.transform(final_data)

## Train the model

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans_2 = KMeans(featuresCol='scaled_features', k=2)
kmeans_3 = KMeans(featuresCol='scaled_features', k=3)

model_2 = kmeans_2.fit(final_data)
model_3 = kmeans_3.fit(final_data)

print('WSSSE_2: {}'.format(model_2.summary.trainingCost))
print('WSSSE_3: {}'.format(model_3.summary.trainingCost))

In [ ]:
## Transform and check if there exists any sample in third cluster (look at the distribution)

from pyspark.sql.functions import count

#model.transform(final_data).select('prediction').show()

model_2.transform(final_data).groupBy('prediction').count().show()
model_3.transform(final_data).groupBy('prediction').count().show()